# I will build a Neural Network for Bank Note Prediction now.

This is not a very challenging deep learning problem. But just for learning.

In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torch.optim as optim
from torchmetrics import Accuracy

import os
import pandas as pd

import pytorch_lightning as pl
from pytorch_lightning.callbacks.progress import TQDMProgressBar
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

from sklearn.model_selection import train_test_split

from torchsummary import summary

In [2]:
# Divide dataset into test and train dataset
dataset = pd.read_csv("BankNote_Authentication.csv")

# Features
X = dataset.iloc[:, :-1].values

# Targets
y = dataset.iloc[:, -1].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [3]:
class BankDataset(Dataset):
    def __init__(self, X, y):
        self.features = X
        self.targets = y

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return torch.Tensor(self.features[idx]), torch.tensor(self.targets[idx], dtype=torch.float32)   #.unsqueeze(0)   #torch.Tensor(self.targets[idx]).long()
        #return self.features[idx], torch.tensor(self.targets[idx]).to(dtype=torch.long)

train_dataset = BankDataset(X_train, y_train)
test_dataset = BankDataset(X_test, y_test)

print("Number train samples", len(train_dataset))
print("Number test samples", len(test_dataset))

Number train samples 1234
Number test samples 138


In [4]:
# dataloaders for training and testing
trainloader = DataLoader(train_dataset, shuffle=True, batch_size=8)
testloader = DataLoader(test_dataset, shuffle=True, batch_size=10)

In [5]:
a = iter(trainloader)

feat, targ = next(a)

print(feat.shape)
print(targ.shape)

torch.Size([8, 4])
torch.Size([8])


### Defining the model

In [6]:
class BankModel(nn.Module):
    def __init__(self, input_size, output_size, hidden_layers=[24, 16, 8, 4], drop_p=0.5):
        super().__init__()
        # Input to a hidden layer
        self.hidden_layers = nn.ModuleList([nn.Linear(input_size, hidden_layers[0])])
        
        # Add a variable number of more hidden layers
        layer_sizes = zip(hidden_layers[:-1], hidden_layers[1:])
        self.hidden_layers.extend([nn.Linear(h1, h2) for h1, h2 in layer_sizes])
        
        self.output = nn.Linear(hidden_layers[-1], output_size)
        
        self.dropout = nn.Dropout(p=drop_p)
        
        
    def forward(self, x):
        ''' Forward pass through the network, returns the output logits '''
        
        for each in self.hidden_layers:
            x = F.relu(each(x))
            x = self.dropout(x)
        x = self.output(x)
        
        return x

### Ptl module

In [7]:
class BankNoteClassifier(pl.LightningModule):
    def __init__(self, model, learning_rate=1e-5):
        super().__init__()
        self.model = model
        self.learning_rate = learning_rate

        # Criterion
        self.criterion = nn.MSELoss()   #nn.BCELoss()   #nn.CrossEntropyLoss(reduction='sum')

        # Accuracy
        self.val_accuracy = Accuracy()
        self.test_accuracy = Accuracy()

    def forward(self, x):
        x = self.model(x)
        return x    #F.log_softmax(x, dim=1)

    def training_step(self, batch, batch_idx):
        oscillation, label = batch
        
        logits = self.forward(oscillation)
        loss = self.criterion(logits, label)
        self.log("train_loss", loss, prog_bar=True, logger=True)
        return loss

    def validation_step(self, batch, batch_idx):
        oscillation, label = batch

        logits = self.forward(oscillation)
        loss = self.criterion(logits, label)
        preds = F.log_softmax(logits, dim=1).argmax(dim=1)
        self.val_accuracy.update(preds, label)

        # Calling self.log will surface up scalars for you in TensorBoard
        self.log("val_loss", loss, prog_bar=True, logger=True)
        self.log("val_acc", self.val_accuracy, prog_bar=True)

        return loss
    
    def test_step(self, batch, batch_idx):
        oscillation, label = batch

        logits = self.forward(oscillation)
        loss = self.criterion(logits, label)
        preds = F.log_softmax(logits, dim=1).argmax(dim=1)
        self.test_accuracy.update(preds, label)

        # Calling self.log will surface up scalars for you in TensorBoard
        self.log("test_loss", loss, prog_bar=True, logger=True)
        self.log("test_acc", self.test_accuracy, prog_bar=True)

        return loss

    def configure_optimizers(self):
        return optim.Adam(self.model.parameters(), lr=self.learning_rate)

In [8]:
net = BankModel(input_size=4, output_size=1)

In [9]:
print(net)

BankModel(
  (hidden_layers): ModuleList(
    (0): Linear(in_features=4, out_features=24, bias=True)
    (1): Linear(in_features=24, out_features=16, bias=True)
    (2): Linear(in_features=16, out_features=8, bias=True)
    (3): Linear(in_features=8, out_features=4, bias=True)
  )
  (output): Linear(in_features=4, out_features=1, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)


In [10]:
# Defining Callbacks
checkpoint_callback = ModelCheckpoint(
    dirpath="training_output",
    filename="best-checkpoint",
    save_top_k = 2,
    verbose = True,
    monitor = "val_loss",
    mode = "min"
)

# Log to Tensor Board
logger = TensorBoardLogger("lightning_logs", name = "bank-predict")

# Stop trainining if model is not improving
early_stopping_callback = EarlyStopping(monitor = "train_loss", patience = 50)

# Progress bar
progress_bar = TQDMProgressBar(refresh_rate=1)

# Model
model = BankNoteClassifier(net, learning_rate=1e-5)

# Defining a Pytorch Lightning Trainer
N_EPOCHS = 50
trainer = pl.Trainer(
    logger = logger,
    enable_progress_bar=True,
    log_every_n_steps=2,
    callbacks = [early_stopping_callback, early_stopping_callback, progress_bar],
    max_epochs = N_EPOCHS,
    accelerator='gpu',
    )

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [11]:
# train model
trainer.fit(model, train_dataloaders=trainloader)

/home/ibad/venvs/deeply/lib/python3.10/site-packages/pytorch_lightning/trainer/configuration_validator.py:117: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type      | Params
--------------------------------------------
0 | model         | BankModel | 697   
1 | criterion     | MSELoss   | 0     
2 | val_accuracy  | Accuracy  | 0     
3 | test_accuracy | Accuracy  | 0     
--------------------------------------------
697       Trainable params
0         Non-trainable params
697       Total params
0.003     Total estimated model params size (MB)
/home/ibad/venvs/deeply/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:219: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number 

Training: 0it [00:00, ?it/s]

/home/ibad/venvs/deeply/lib/python3.10/site-packages/torch/nn/modules/loss.py:530: UserWarning: Using a target size (torch.Size([8])) that is different to the input size (torch.Size([8, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/home/ibad/venvs/deeply/lib/python3.10/site-packages/torch/nn/modules/loss.py:530: UserWarning: Using a target size (torch.Size([2])) that is different to the input size (torch.Size([2, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/home/ibad/venvs/deeply/lib/python3.10/site-packages/pytorch_lightning/trainer/trainer.py:653: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [ ]:
#trainer.test(testloader)